# Description

#### Box Circle - 8/22/2019

BCC revenue for this current FY: One for membership donations (including new and renewals), the other for ticket sales please?

Also could you find out what the “goal” (finger quotes) is? 


In [1]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [2]:
import pandas as pd

In [3]:
path_data = 'U:/data'

# Globals

In [4]:
FY_START = 20
FY_END = 20

In [5]:
d_files = 'donors_fy13-present.csv'

In [6]:
def donor_data_import(path_to_data_files, file):
    donor_raw = pd.read_csv(path_to_data_files + '/donors/' + file, encoding='ISO-8859-1')
    return donor_raw

In [7]:
d_data = donor_data_import(path_data, d_files)

In [8]:
donor_fy = d_data.campaign.str[3:8] # parse last 5 characters of campaign column
d_data['fy'] = donor_fy # append that to a column named 'fy'

In [9]:
filter_fys = {fy: False for fy in d_data.fy.drop_duplicates()}
for fy,boo in filter_fys.items():
    try:
        int(fy[0])
        if int(fy[3:]) == FY_START:
            filter_fys[fy] = True
        else:
            continue
    except:
        continue

In [10]:
d_data['keep'] = d_data['fy'].map(filter_fys)
donor_df = d_data.loc[d_data.keep].reset_index()
donor_df.drop(columns=['keep'], inplace=True)

In [11]:
donor_df.columns

Index(['index', 'trn_count', 'contribution_amt', 'pledge', 'pledge_payment',
       'gift', 'gift_plus_pledge', 'restricted', 'write_off', 'total_received',
       'batch_no', 'ref_no', 'fund_no', 'fund_desc', 'fyear', 'campaign_no',
       'campaign', 'cont_designation_id', 'cont_designation', 'channel_id',
       'channel_desc', 'pmt_method_id', 'pmt_method', 'pledge_pmt_amt',
       'gift_pmt_amt', 'pmt_count', 'customer_no', 'lname', 'fname',
       'creditee_type', 'creditee_type_desc', 'creditee_no', 'creditee_name',
       'resolved_cust_type', 'resolved_cust_id', 'resolved_cust_name',
       'summary_cust_id', 'summary_cust_name', 'summary_sort', 'display_name',
       'sort_name', 'cont_dt', 'cont_fy', 'cont_fy_month', 'cont_cy',
       'cont_cy_month', 'trn_dt', 'trn_fy', 'trn_fy_month', 'trn_cy',
       'trn_cy_month', 'posted_dt', 'post_fy', 'post_fy_month', 'post_cy',
       'post_cy_month', 'ps_sol', 'res_ps_sol', 'sum_ps_sol', 'gl_no',
       'gl_natural', 'gl_sub_dept',

In [12]:
donor_df.campaign.head()

0    PS 19-20 Capital Campaign
1          PS 19-20 Box Circle
2          PS 19-20 Box Circle
3          PS 19-20 Box Circle
4          PS 19-20 Box Circle
Name: campaign, dtype: object

In [13]:
bcc = donor_df.loc[donor_df['campaign'] == 'PS 19-20 Box Circle']

In [14]:
bcc.groupby(['campaign']).agg({
    'campaign': ['count'],
    'gift': ['sum'],
    'pledge': ['sum'],
    'gift_plus_pledge': ['sum']
})

,campaign,gift,pledge,gift_plus_pledge
,count,sum,sum,sum
campaign,,,,
PS 19-20 Box Circle,295,101500.0,136625.0,238125.0


In [21]:
bcc_20 = bcc['summary_cust_id'].drop_duplicates()

In [22]:
len(bcc_20)

99

# Ticketing

In [26]:
def ticketing_data_import(path_to_data_files, filename):
    df = pd.read_csv(path_data + '/ticketing/' + filename, skiprows=3)
    return df

In [29]:
t_file = 'all_fy20_td.csv'

In [30]:
t_df = ticketing_data_import(path_data, t_file)

In [33]:
t_filtered = t_df[t_df['summary_cust_id'].isin(bcc_20)]

In [34]:
t_filtered.columns

Index(['perf_code', 'perf_no', 'perf_dt', 'zone_no', 'zone_desc', 'section',
       'row', 'seat', 'home_price', 'paid_amt', 'fee_amt', 'seat_status',
       'seat_status_desc', 'customer_no', 'order_no', 'order_dt',
       'price_type_group_id', 'price_type_group', 'pd_up', 'season',
       'season_desc', 'summary_cust_id', 'summary_cust_name', 'attended'],
      dtype='object')

In [37]:
t_filtered.groupby(['season_desc', 'price_type_group']).agg({
    'paid_amt': ['count', 'sum']
})

paid_amt          
                                         count       sum
season_desc          price_type_group                   
PS 19-20 Chamber     Subscription           18    1272.0
PS 19-20 Classics    Single                 21    2922.0
                     Subscription         1690  268566.0
PS 19-20 Connections Subscription           16    1360.0
PS 19-20 Family      Subscription           90    3144.0
PS 19-20 Organ       Comp                   48       0.0
                     Subscription            6     310.0
PS 19-20 Pops        Flex                    8    1592.0
                     Single                 28    4298.0
                     Subscription          540  101559.0
PS 19-20 Specials    Comp                   36       0.0
                     Single                 39    6039.0
PS 2019 Summer       Comp                   34       0.0
                     Single                 67    2862.0
                     Subscription          112    9716.0